In [0]:
# Code pour entrainer la dectetion de disc optic et couper l'image autour du mask.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
debug = False
# debug = True

In [0]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob 

In [0]:
DATA_DIR = '/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Second Branch/mask_RCNN/data_train/'
#DATA_DIR = '/kaggle/retina/idrid/'
# Directory to save logs and trained model
ROOT_DIR = '/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Second Branch/mask_RCNN/'
#ROOT_DIR = '/working/'

### Install Matterport's Mask-RCNN model from github.
See the [Matterport's implementation of Mask-RCNN](https://github.com/matterport/Mask_RCNN).

In [12]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
#!python setup.py -q install

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.84 MiB | 33.89 MiB/s, done.
Resolving deltas: 100% (570/570), done.


In [13]:
1024# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

Using TensorFlow backend.


In [0]:
mask_path = os.path.join(DATA_DIR, "A. Segmentation/2. All Segmentation Groundtruths/a. Training Set/")
image_path = os.path.join(DATA_DIR, "A. Segmentation/1. Original Images/a. Training Set/")
# test_dicom_dir = os.path.join(DATA_DIR, 'test_v2')

In [0]:
#For tensorflow on gpu
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1



### Download COCO pre-trained weights

In [0]:
# !wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
# !ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = "/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Second Branch/mask_RCNN/mask_rcnn_coco.h5"

#COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"

### Some setup functions and classes for Mask-RCNN

- dicom_fps is a list of the dicom image path and filenames 
- image_annotions is a dictionary of the annotations keyed by the filenames
- parsing the dataset returns a list of the image filenames and the annotations dictionary

In [0]:
WIDTH = 1024
LENGTH = 1024
SHAPE = (WIDTH, LENGTH)

In [0]:
# The following parameters have been selected to reduce running time for demonstration purposes 
# These are not optimal 

class DetectorConfig(Config):    
    # Give the configuration a recognizable name  
    NAME = 'airbus'
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2  # background and ship classes
    
    IMAGE_MIN_DIM = WIDTH
    IMAGE_MAX_DIM = WIDTH
    RPN_ANCHOR_SCALES = (16, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 64
#     MAX_GT_INSTANCES = 14
#     DETECTION_MAX_INSTANCES = 10
    DETECTION_MIN_CONFIDENCE = 0.75
#     IMAGE_RESIZE_MODE = "crop"
    DETECTION_NMS_THRESHOLD = 0.0

    STEPS_PER_EPOCH = 15 if debug else 150
    VALIDATION_STEPS = 10 if debug else 125
    
    ## balance out losses
    LOSS_WEIGHTS = {
        "rpn_class_loss": 0.1,
        "rpn_bbox_loss": 0.1,
        "mrcnn_class_loss": 0.1,
        "mrcnn_bbox_loss": 0.1,
        "mrcnn_mask_loss": 0.1
    }

config = DetectorConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.75
DETECTION_NMS_THRESHOLD        0.0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 0.1, 'rpn_bbox_loss': 0.1, 'mrcnn_class_loss': 0.1, 'mrcnn_bbox_loss': 0.1, 'mrcnn_mask_loss': 0.1}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INST

In [0]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from skimage.morphology import binary_opening, disk, label
import gc; gc.enable() # memory is tight

montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)

def multi_rle_encode(img, **kwargs):
    '''
    Encode connected regions as separated masks
    '''
    labels = label(img)
    if img.ndim > 2:
        return [rle_encode(np.sum(labels==k, axis=2), **kwargs) for k in np.unique(labels[labels>0])]
    else:
        return [rle_encode(labels==k, **kwargs) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img, min_max_threshold=1e-3, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    if np.max(img) < min_max_threshold:
        return '' ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return '' ## ignore overfilled mask
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.uint8)
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks |= rle_decode(mask)
    print("masks_as_image, ", end="")
    return all_masks

def masks_as_color(in_mask_list):
    # Take the individual ship masks and create a color mask array for each ships
    all_masks = np.zeros((768, 768), dtype = np.float)
    scale = lambda x: (len(in_mask_list)+x+1) / (len(in_mask_list)*2) ## scale the heatmap image to shift 
    for i,mask in enumerate(in_mask_list):
        if isinstance(mask, str):
            all_masks[:,:] += scale(i) * rle_decode(mask)
    return all_masks

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# function to resize the image keeping the height proportional to the width.
def resizeAndPad(img, size, padColor=0):
    h, w = img.shape[:2]
    sh, sw = size

    # interpolation method
    if h > sh or w > sw: # shrinking image
        interp = cv2.INTER_AREA

    else: # stretching image
        interp = cv2.INTER_CUBIC

    # aspect ratio of image
    aspect = float(w)/h
    saspect = float(sw)/sh

    if (saspect > aspect) or ((saspect == 1) and (aspect <= 1)):  # new horizontal image
        new_h = sh
        new_w = np.round(new_h * aspect).astype(int)
        pad_horz = float(sw - new_w) / 2
        pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
        pad_top, pad_bot = 0, 0

    elif (saspect < aspect) or ((saspect == 1) and (aspect >= 1)):  # new vertical image
        new_w = sw
        new_h = np.round(float(new_w) / aspect).astype(int)
        pad_vert = float(sh - new_h) / 2
        pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
        pad_left, pad_right = 0, 0

    # set pad color
    if len(img.shape) is 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided
        padColor = [padColor]*3

    # scale and padBORDER_CONSTANT, value=padColor)

    # scale and pad
    scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
    scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

    return scaled_img

In [0]:
import numpy as np
import argparse
import cv2
import imutils

# Two fonction for create binary mask for keras
def imshow_components(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, labels = cv2.connectedComponents(gray)
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    labeled_img[label_hue==0] = 0
    return (labeled_img)

import numpy as np
import pandas as pd
from skimage.io import imread
import matplotlib.pyplot as plt

def mask_fct(img):
    img = imshow_components(img)
    colors = np.unique(img.reshape(-1, img.shape[-1]), axis=0)
    colors = colors[1:]
    masks = None
    save_img = img.copy()
    for index, color in enumerate(colors):
        img = save_img.copy()
        mask = (img == color).all(axis=2)
        img[np.invert(mask)]=[0,0,0]
        img = img.astype(bool)
        mask = np.logical_or.reduce(img, axis=2) 
        if index == 0:
            masks = mask.copy()
            masks = np.reshape(masks, (WIDTH, LENGTH, 1))
        else:
            masks = np.dstack((masks, mask))
    class_ids = np.zeros((len(colors)), dtype=np.int32)
    class_ids[:] = 1
    return(masks, class_ids)

In [0]:
def create_tab_mask(path):
    mask = cv2.imread(path)
    mask = resizeAndPad(mask, SHAPE)
    tab_mask = mask_fct(mask)
    return (tab_mask)

In [0]:
def rename(row):
    if "flip" in row:
        return (row[:-12] + "_flip.jpg")
    return (row[:-7] + ".jpg")

In [0]:
def save_dataframe(ann):
    ann['Mask_1'] = ann['Mask_1'].apply(lambda row: str(list(row.astype(int).reshape(row.shape[0]*row.shape[1]*row.shape[2]))))
    ann['Mask_2'] = ann['Mask_2'].apply(lambda row: np.array2string(row, separator=','))
    ann.to_excel("anns.xlsx")
    print(ann.Mask_2.iloc[1])
    print("file created and saved")

In [0]:
#create dataframe for pathology selected
files = os.listdir(mask_path)
files = files[2:3]
images = []
for path in files:
    path = os.path.join(mask_path, path)
    paths = os.listdir(path)
    for img in paths:
        img = os.path.join(path, img)
        images.append(img)
anns = pd.DataFrame(images, columns=['Paths_mask'])
anns['ID_mask'] = anns['Paths_mask'].apply(lambda row: row.split(os.path.sep)[-1])
anns["ID"] = anns["ID_mask"].apply(lambda row: rename(row))
anns["Path"] = anns["ID"].apply(lambda row: image_path + row)

In [0]:
# Split dataframe 80/20 train/test
train_names = anns.ID.unique().tolist()  ## override with ships

test_size = config.VALIDATION_STEPS * config.IMAGES_PER_GPU
image_fps_train, image_fps_val = train_test_split(train_names, test_size=0.2, random_state=42)

if debug == 2:
    image_fps_train = image_fps_train[:100]
    image_fps_val = image_fps_val[:100]
    test_names = test_names[:100]

print(len(image_fps_train), len(image_fps_val))

120 30


In [0]:
import cv2
class DetectorDataset(utils.Dataset):
    """Dataset class for training our dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # Add classes
        self.add_class('point', 1, 'point')
        # add images 
        for i, fp in enumerate(image_fps):
            annotations = image_annotations.query('ID =="' + fp + '"')['Paths_mask'].iloc[0]
            self.add_image('point', image_id=i, path=os.path.join(image_path, fp), 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
                            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        image = imread(fp)
        image = resizeAndPad(image, SHAPE)
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        mask = cv2.imread(info['annotations'])
        mask = resizeAndPad(mask, SHAPE)
        tab_mask = mask_fct(mask)
        return (tab_mask)

### Examine the annotation data, parse the dataset, and view dicom fields

In [0]:
image_fps, image_annotations = train_names, anns

In [0]:
ORIG_SIZE = WIDTH
ORIG_SIZE

1024

### Create and prepare the training dataset using the DetectorDataset class.

In [0]:
%%time
# prepare the training dataset
dataset_train = DetectorDataset(image_fps_train, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_train.prepare()
print(len(image_fps_train))

120
CPU times: user 212 ms, sys: 2.13 ms, total: 214 ms
Wall time: 230 ms


In [0]:
%%time
# prepare the validation dataset
dataset_val = DetectorDataset(image_fps_val, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_val.prepare()

CPU times: user 52.6 ms, sys: 4.86 ms, total: 57.5 ms
Wall time: 54.5 ms


### Now it's time to train the model. Note that training even a basic model can take a few hours. 

Note: the following model is for demonstration purpose only. We have limited the training to one epoch, and have set nominal values for the Detector Configuration to reduce run-time. 

- dataset_train and dataset_val are derived from DetectorDataset 
- DetectorDataset loads images from image filenames and  masks from the annotation data
- model is Mask-RCNN

In [0]:
import mrcnn.model as modellib
from mrcnn.config import Config

model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

# Exclude the last layers because they require a matching
# number of classes
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


OSError: ignored

In [0]:
model

In [0]:
LEARNING_RATE = 0.0001

# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")

In [0]:
%%time
model.train(dataset_train, dataset_val,
            learning_rate=LEARNING_RATE,
            epochs=4 if debug else 250,
            layers='heads',
            augmentation=augmentation)


In [0]:
history = model.keras_model.history.history

In [0]:
epochs = range(1, len(history['loss'])+1)
pd.DataFrame(history, index=epochs)

In [0]:
plt.figure(figsize=(21,11))

plt.subplot(231)
plt.plot(epochs, history["loss"], label="Train loss")
plt.plot(epochs, history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(232)
plt.plot(epochs, history["rpn_class_loss"], label="Train RPN class ce")
plt.plot(epochs, history["val_rpn_class_loss"], label="Valid RPN class ce")
plt.legend()
plt.subplot(233)
plt.plot(epochs, history["rpn_bbox_loss"], label="Train RPN box loss")
plt.plot(epochs, history["val_rpn_bbox_loss"], label="Valid RPN box loss")
plt.legend()
plt.subplot(234)
plt.plot(epochs, history["mrcnn_class_loss"], label="Train MRCNN class ce")
plt.plot(epochs, history["val_mrcnn_class_loss"], label="Valid MRCNN class ce")
plt.legend()
plt.subplot(235)
plt.plot(epochs, history["mrcnn_bbox_loss"], label="Train MRCNN box loss")
plt.plot(epochs, history["val_mrcnn_bbox_loss"], label="Valid MRCNN box loss")
plt.legend()
plt.subplot(236)
plt.plot(epochs, history["mrcnn_mask_loss"], label="Train Mask loss")
plt.plot(epochs, history["val_mrcnn_mask_loss"], label="Valid Mask loss")
plt.legend()

plt.show()

In [0]:
fbest_epoch = np.argmin(history["val_loss"])
score = history["val_loss"][best_epoch]
print(f'Best Epoch:{best_epoch+1} val_loss:{score}')

In [0]:
# select trained model 
dir_names = next(os.walk(model.model_dir))[1]
print (dir_names)
#best_epoch = 149
dir_names = [dir_names[-1]]
dir_names = ["optic_disc"]
print(dir_names)

if not dir_names:
    import errno
    raise FileNotFoundError(
        errno.ENOENT,
        "Could not find model directory under {}".format(self.model_dir))

fps = []
# Pick last directory
#print (best_epoch)
for d in dir_names: 
    print("d = ", d)
    dir_name = os.path.join(model.model_dir, d)
    print(dir_name)
    # Find the last checkpoint
    #print (next(os.walk(dir_name)))
    checkpoints = next(os.walk(dir_name))[2]
    checkpoints = filter(lambda f: f.startswith("optic"), checkpoints)
    checkpoints = sorted(checkpoints)
    print ("checkpoint : ", checkpoints)
    if not checkpoints:
        print('No weight files in {}'.format(dir_name))
    else:
        print ("dir = ", dir_name)
        checkpoint = os.path.join(dir_name, checkpoints[best_epoch])
        fps.append(checkpoint)

print("fps : ", fps)
model_path = sorted(fps)[-1]
print('Found model {}'.format(model_path))

In [0]:
class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=ROOT_DIR)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [0]:
# set color for class
def get_colors_for_class_ids(class_ids):
    colors = []
    for class_id in class_ids:
        if class_id == 1:
            colors.append((.941, .204, .204))
    return colors

In [0]:
# Image augmentation (light but constant)
aug_detect = iaa.Sequential([
    iaa.CLAHE()
])

In [0]:
def save_name(name, aug, epoch):
    augm = "_CLAHE" if aug else ""
    name = name[:-4]
    name = "save_day/" + name + "_ep" + str(epoch+1) + augm
    return (name)

In [0]:
#from visualize import *

In [0]:
from mrcnn import visualize

In [0]:
# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 1 + 80

# path folder for testing images
path = "/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Second Branch/mask_RCNN/Input_output/input/"
images = os.listdir(path)
i=0
for name in images:
    img = imread(path+name)
    img_detect = img.copy()
    print(name)
    aug = False
    if aug:
        img_detect = aug_detect(image=img)
    # make prediction
    results = model.detect([img_detect], verbose=1)
    # visualize the results
    r = results[0]
    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, r['scores'],
                     )
    y, x, h, w = r['rois'][0]
    #cropped_image = tf.image.crop_to_bounding_box(img, y-30, x-30, y-(y-30) , x-(x-30))
    #print(cropped_image)
    #plt.imshow(cropped_image)
    #crop_img = img[y:y+h, x:x+w]
    #plt.imshow("cropped", crop_img)
    #cv2.imread(cropped_image)
    y, x = y-30, x-30
    h, w = h+30, w+30
    roi = img[y:h, x:w,:]
    plt.imshow(roi)
    cv2.imwrite("/content/drive/My Drive/CRP_Attali_Wu_Allouche_Bekdouche/Second Branch/output_rcnn/roi{0}.png".format(i), cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))
    i=i+1

ValueError: ignored

In [0]:
cwd = os.getcwd()

In [0]:
import os
print( os.getcwd() )
#print( os.listdir('roi.png') )

In [0]:
print(dataset_val.class_names)